In [18]:
import json
import requests
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import pymysql

# Define the SQLAlchemy model
Base = declarative_base()

C:\Users\dovil\AppData\Local\Temp\ipykernel_15652\550956766.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [24]:
import json
import requests
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# Define the SQLAlchemy model
Base = declarative_base()

class Breed(Base):
    __tablename__ = 'Breeds'
    id = Column(Integer, primary_key=True)
    breed = Column(String(30))
    origin = Column(String(50))

class BreedImage(Base):
    __tablename__ = 'Breeds_img'
    id = Column(Integer, primary_key=True)
    url = Column(String(50))
    img = Column(String(50))

# Create the database engine and session
engine = create_engine('mysql+pymysql://root@localhost/test',  pool_recycle=3600)
Session = sessionmaker(bind=engine)
session = Session()

# Create the tables
Base.metadata.create_all(engine)


C:\Users\dovil\AppData\Local\Temp\ipykernel_15652\305910917.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [25]:
# Make the API request
url = "https://dog-breeds2.p.rapidapi.com/dog_breeds/group/breed_status/extinct"
headers = {
    "X-RapidAPI-Key": "51b07a06c6mshbaae7fd65806f5cp13da34jsn59c5aa35baa7",
    "X-RapidAPI-Host": "dog-breeds2.p.rapidapi.com"
}
response = requests.get(url, headers=headers)
api_data = response.json()

# Insert data into the tables
for record in api_data:
    breed = Breed(id=record['id'], breed=record['breed'], origin=record['origin'])
    session.add(breed)
    breed_image = BreedImage(id=record['id'], url=record['url'], img=record['img'])
    session.add(breed_image)

session.commit()

In [27]:
# Query the data using SQLAlchemy
breeds = session.query(Breed.breed, Breed.origin, BreedImage.url, BreedImage.img).\
    join(BreedImage, Breed.id == BreedImage.id).all()

# Close the session
session.close()

# Convert the query result to a Pandas DataFrame
import pandas as pd
df = pd.DataFrame(breeds, columns=['Breed', 'Origin', 'Link', 'Img'])
df.head()

,Breed,Origin,Link,Img
0,Old English Terrier,England,https://en.wikipedia,https://upload.wikim
1,African hairless dog,Africa,https://en.wikipedia,https://upload.wikim
2,Alaunt,Pontic-Caspian Stepp,https://en.wikipedia,https://upload.wikim
3,Alpine Mastiff,Swiss and Italian Al,https://en.wikipedia,https://upload.wikim
4,Belgian Mastiff,Belgium and the Neth,https://en.wikipedia,https://upload.wikim
